In [2]:
import json
import mysql.connector
import pandas as pd

from IPython.display import display, Markdown, HTML

db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="secret",
    port="3306",
    database="serlo",
    charset="latin1"
)

def query(sql):
    c = db.cursor()
    c.execute(sql)
    
    return c.fetchall()

def querySingleton(sql):
    return [ x[0] for x in query(sql) ]

In [35]:
def create_community_walls(usernames):
    display(Markdown("# Icon-Größe: 50px"))
    create_community_wall(usernames, size=50, margin=5)
    
    
    display(Markdown("# Icon-Größe: 75px"))
    create_community_wall(usernames, size=75, margin=7)
    
    
    display(Markdown("# Icon-Größe: 100px"))
    create_community_wall(usernames, size=100)

def create_community_wall(usernames, size=75, margin=10):
    result = """<div style="display: flex; flex-wrap: wrap;
                            justify-content: space-around;
                            align-items: center;">"""
    
    for username in usernames:
        result += f"""
            <img src="https://community.serlo.org/avatar/{username}"
                 style="border-radius: 50%; width: {size}px;
                        height: {size}px; display: block;
                        margin: {margin}px;"/>
        """
    
    result += "</div>"
    
    display(HTML(result))

results = query("""
    select max(user.username), count(event_log.actor_id) as count from event_log
    join user on user.id = event_log.actor_id
    where event_log.date > Date("2020-09-29")
    group by event_log.actor_id
    order by count desc
""")

display(Markdown("# Community-Walls aller Autor:innen mit mehr als 10 Bearbeitungen im letzten Jahr"))
create_community_walls([x[0] for x in results if x[1] > 10])

# Community-Walls aller Autor:innen mit mehr als 10 Bearbeitungen im letzten Jahr

# Icon-Größe: 50px

# Icon-Größe: 75px

# Icon-Größe: 100px

In [37]:
from getpass import getpass

token = getpass("Rocket-Chat Token")
user_id = getpass("Rocket-Chat User-Id")

Rocket-Chat Token········
Rocket-Chat User-Id········


In [57]:
import requests

def list_users(offset = 0):
    result = requests.get("http://community.serlo.org/api/v1/users.list", headers={
        "X-Auth-Token": token,
        "X-User-Id": user_id
    }, params={"count": 100, "offset": offset}).json()
    
    usernames = [x["username"] for x in result["users"]]
    
    if result["total"] > len(usernames) + offset:
        usernames += list_users(offset=len(usernames) + offset)
    
    return usernames

def has_avatar(username):
    req = requests.get(f"https://community.serlo.org/avatar/{username}")
    
    return req.headers["content-type"] != "image/svg+xml"

display(Markdown("# Nur Personen, die ein Avatar haben"))
avatar_users = [username for username in list_users() if has_avatar(username)]
create_community_walls(avatar_users)

# Nur Personen, die ein Avatar haben

# Icon-Größe: 50px

# Icon-Größe: 75px

# Icon-Größe: 100px

In [63]:
results = query("""
    select max(user.username), count(event_log.actor_id) as count from event_log
    join user on user.id = event_log.actor_id
    where event_log.date > Date("2020-09-29")
    group by event_log.actor_id
    having count > 100
    order by count desc
""")

users = set([x[0] for x in results] + avatar_users)
display(Markdown("# Autor:innen mit Bilder oder mit mindestens 100 Bearbeitungen im letzten Jahr"))
create_community_walls(users)

# Autor:innen mit Bilder oder mit mindestens 100 Bearbeitungen im letzten Jahr

# Icon-Größe: 50px

# Icon-Größe: 75px

# Icon-Größe: 100px